# Extract the keywords from the  text of the publications

In [1]:
import pandas as pd
import nltk

In [2]:
data_path = '/home/benjamin/Documents/memetracker/Data/csv/'
pickle_data_path = '/home/benjamin/Documents/memetracker/Data/pickle/'
series_name = 'baron_noir'
csvfile = data_path+series_name+'.csv'
Dataf = pd.read_csv(csvfile,sep=';')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# chargement des stopwords français
french_stopwords = set(stopwords.words('french'))
# add custom stop words
french_stopwords.update(['les','cette','http','https'])

In [ ]:
french_stopwords

In [4]:
# Extract a subset of the data
filtered_data = Dataf[["date","text","title", "platform","hashtags"]].copy()
# the entries of the date column are dates:
filtered_data['date'] = pd.to_datetime(filtered_data['date'])#,infer_datetime_format=True)

In [5]:
# Twitter filter
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tokens_tw = tknzr.tokenize(filtered_data.text[1])
print(filtered_data.text[1])
#print(tokens_tw)
#[word for word in tokens_tw if not 'http' in word]
[token for token in tokens_tw if ((token.lower() not in french_stopwords) 
                                                              and (len(token)>2)
                                                             and not 'http' in token)]

Le Baron Noir, plus grande BD politique de tous les temps : http://t.co/P8JfVQ66HB


['Baron', 'Noir', 'plus', 'grande', 'politique', 'tous', 'temps']

In [32]:
def filter_text(dataframe):
    # filtering the texts
    texts = filtered_data.text
    filtered_text_list=texts.copy()
    for idx,row in dataframe.iterrows():
        #print(ind)
        texte = row.text
        platform = row.platform
        if (not pd.isnull(texte)):
            if platform == 'Twitter':
                tokens_tw = tknzr.tokenize(texte)
                #texte_tw = " ".join( tokens_tw )
                #texte_tw = re.findall(r"\w+", texte_tw)
                #texte_tw = " ".join( texte_tw )
                #tokens_fr = nltk.tokenize.word_tokenize(texte_tw, language='french')
                tokens_ns = [token for token in tokens_tw if ((token.lower() not in french_stopwords) 
                                                              and (len(token)>2)
                                                             and ('http' not in token)
                                                             and ('\'' not in token))]
                filtered_text = " ".join( tokens_ns )
            else:
                # filter out the chars that are not alphabet letters
                texte = re.findall(r"\w+", texte)
                texte = " ".join( texte )
                # Cut the text in tokens
                tokens_fr = nltk.tokenize.word_tokenize(texte, language='french')
                # filter out the french stop words
                # and
                # filter out the tokens smaller than 3 letters
                tokens_ns = [token for token in tokens_fr if ((token.lower() not in french_stopwords) 
                                                              and (len(token)>2))]
                filtered_text = " ".join( tokens_ns )
        else:
            filtered_text = " "
        filtered_text_list[idx] = filtered_text 
    return filtered_text_list

In [33]:
filtered_texts = filter_text(filtered_data)
filtered_data.loc[:,'filtered_text'] = filtered_texts

In [34]:
filtered_data.to_pickle(pickle_data_path+series_name+'_texts'+'.pkl')

In [35]:
filtered_data.filtered_text[2:50]

2                                      Baron Noir suite
3     Excellent teeeellement vrai @OLecointe Baron N...
4                       @Pward13 Baron Noir plus classe
5     Situé route Sharm Sheikh Dahab saura charmer a...
6     Peux dire plus étrier Montage réglages peux pa...
7     maertens écrit Peux dire plus étrier Montage r...
8     confirme Statistiques Posté rickyfirst Jeu Jan...
9     maertens écrit Peux dire plus étrier Montage r...
10    Merci gars Statistiques Posté maertens Jeu Jan...
11    maertens écrit Peux dire plus étrier Montage r...
12    Paroles chanson baron noir chantée Arthur baro...
13    Billet PASCAL PORCHER CROISIERE BARON NOIR SOU...
14    Discussion publiée PASCAL PORCHER CROISIERE SO...
15    CROISIERE BARON NOIR SOUDAN FEVRIER 2015 URGEN...
16    @scuba_people CROISIERE BARON NOIR SOUDAN FEVR...
17    Bonjour tous voudrais mettre leviers bout prol...
18         Nouveau coup coeur Arthur Baron Noir #deezer
19    Top Ten Tuesday TTT donc rendez hebdomadai

## Extracting most frequent words with sci-kit learn

In [36]:
# loading the data as a list to be processed by sci-kit learn programs
clean_train_reviews = []
for text in filtered_data.filtered_text:
    clean_train_reviews.append(text)

In [37]:
import sklearn
import numpy as np

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [39]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print('Nb of documents: '+str(train_data_features.shape[0])+', '+
      'Nb of features: '+str(train_data_features.shape[1]))

Creating the bag of words...

Nb of documents: 29471, Nb of features: 5000


In [40]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

vocabdf = pd.DataFrame()
vocabdf['words'] = vocab
vocabdf['count'] = dist
vocabdf = vocabdf.sort_values('count',ascending=False)
vocabdf = vocabdf.reset_index(drop=True)
# Saving the data
vocabdf.to_pickle(pickle_data_path+series_name+'_vocab_bow'+'.pkl')
vocabdf.loc[0:50,:]

,words,count
0,noir,19132
1,baron,18939
2,série,16201
3,baronnoir,15575
4,plus,10725
5,politique,9174
6,canal,9037
7,casting,6580
8,2016,6385
9,tout,5881


In [41]:
print("Creating the TFIDF vectors...\n")
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features_tfidf = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features_tfidf = train_data_features_tfidf.toarray()
vocab_tfidf = vectorizer.get_feature_names()
print('Nb of documents: '+str(train_data_features_tfidf.shape[0])+', '+
      'Nb of features: '+str(train_data_features_tfidf.shape[1]))

Creating the TFIDF vectors...

Nb of documents: 29471, Nb of features: 5000


In [42]:
# Sum up the counts of each vocabulary word
dist_tfidf = np.sum(train_data_features_tfidf, axis=0)

vocabdf = pd.DataFrame()
vocabdf['words'] = vocab_tfidf
vocabdf['count'] = dist_tfidf
vocabdf = vocabdf.sort_values('count',ascending=False)
vocabdf = vocabdf.reset_index(drop=True)
# Saving the data
vocabdf.to_pickle(pickle_data_path+series_name+'_vocab_tfidf'+'.pkl')
vocabdf.loc[0:50,:]

,words,count
0,baronnoir,2342.811153
1,noir,2058.330993
2,baron,2056.237946
3,série,1358.085107
4,canal,941.503291
5,politique,933.103956
6,canalplus,870.112593
7,kad,676.008663
8,merad,621.625339
9,saison,610.867047


In [43]:
for text in filtered_data.filtered_text:
    if '\'un ' in text:
        print(text)

In [20]:
french_stopwords

{'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'cette',
 'd',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'http',
 'https',
 'il',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes',
 'moi',
 'mon',
 'même',
 'n',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ont',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 's',
 'sa',
 'se',
 'sera',
 'serai',
 'seraient',
 